# Distance Computation

A script to compute the distance from each hand-joint to the object, based on the pose data.


#### Notes

Input:

- Hand Pose: 1 (annotate?) + $21*3$ (x,y,z in order) (left hand) + 1 + $21*3$ (right hand)
- Object Pose: 1 (object class) + $21*3$ (x,y,z in order) bounding box (1 center, 8 corners, 12 mid-edge)

Output:

- Distance map: 21 (left hand) + 21 (right hand) distances from joints

----

Imports

In [21]:
import os
import numpy as np
from numpy.typing import NDArray
import tqdm

Parameters

In [22]:
n_frames_per_seq : int = 8
mode = 'train' # 'train' , 'val' , 'test'
h2o_root = '../data/h2o/'

Load Pose Sequences

In [23]:
action_labels = np.load(h2o_root + f'action_labels_{mode}.npy')
n_sequences = action_labels.shape[0]
print(f"Loaded {n_sequences} action sequences for {mode}.")

Loaded 569 action sequences for train.


In [24]:
def line_plane_intersection(line_pt: NDArray, line_dir: NDArray, plane_vertices: NDArray):
    """
    Find the intersection, if any, of a line and a plane defined by 4 vertices.

    Parameters
    ----------
    line_pt : NDArray (3,)
        A point on the line.
    line_dir : NDArray (3,)
        The direction of the line.
    plane_vertices : NDArray (4,3)
        4 vertices defining the plane.
    
    Returns
    -------
    NDArray (3,) or None
        The intersection point or None if the line is parallel to the plane.
    """
    assert line_pt.shape == (3,)
    assert line_dir.shape == (3,)
    assert plane_vertices.shape == (4,3)

    plane_normal = np.cross(plane_vertices[1] - plane_vertices[0], plane_vertices[3] - plane_vertices[0])
    plane_norm = np.linalg.norm(plane_normal)
    assert plane_norm > 1e-6
    plane_normal /= plane_norm
    
    denom = np.dot(line_dir, plane_normal)
    if denom < 1e-6:
        # debug print TODO
        # print("| No I |", end="")
        return None # line is parallel to plane
    
    t = np.dot(plane_vertices[0] - line_pt, plane_normal) / denom
    intersection = line_pt + t * line_dir
    assert intersection.shape == (3,)
    return intersection


In [33]:
def is_pt_in_rect(pt: NDArray, rect_vertices: NDArray) -> bool:
    """
    Check whether a point is within a rectangle defined by the 4 vertices.

    Parameters
    ----------
    pt : NDArray (3,)
        The point to check.
    rect_vertices : NDArray (4, 3)
        The vertices of the rectangle, such that succeeding vertices are connected.

    Returns
    -------
    bool
        Whether the point is within the rectangle.


    Algorithm:
    Take 3 consecutive vertices ABC of the rectangle and the sample point P.
    P is inside the rectangle, iff 
    0 <= dot(AB, AP) <= dot(AB, AB) and 
    0 <= dot(BC, BP) <= dot(BC, BC)
    """
    assert pt.shape == (3,)
    assert rect_vertices.shape == (4, 3)
    tol = np.double(1e-2)

    # compute AB, BC
    edge1 = rect_vertices[1] - rect_vertices[0]
    edge2 = rect_vertices[2] - rect_vertices[1]
    # compute AP, BP
    vec1 = pt - rect_vertices[0]
    vec2 = pt - rect_vertices[1]

    return -tol <= np.dot(edge1, vec1) <= np.dot(edge1, edge1) + tol and -tol <= np.dot(edge2, vec2) <= np.dot(edge2, edge2) + tol


    # dennis
    # total_angle = 0
    # for i in range(rect_vertices.shape[0]):
    #     a, b = rect_vertices[i], rect_vertices[(i + 1) % len(rect_vertices)]
    #     da, db = a - pt, b - pt
    #     angle = np.arctan2(np.linalg.norm(np.cross(da, db)), np.dot(da, db))
    #     total_angle += angle
    # return np.isclose(total_angle, 2 * np.pi, atol=1e-5)

    

In [26]:
def dist_joint_bbox(joint: NDArray, center: NDArray, faces: NDArray):
    """
    Compute the distance from a joint to the bounding box of an object.

    Parameters
    ----------
    joint : NDArray (3,)
        The joint position.
    center : NDArray (3,)
        The object's center.
    faces : List[NDArray (4,3)]
        The faces of the object's bounding box.
    
    Returns
    -------
    float
        The minimum distance from the joint to the bounding box.

    Exceptions
    ----------
    AssertionError
        If no distance is found.
    """
    assert joint.shape == (3,)
    assert center.shape == (3,)
    assert faces[0].shape == (4, 3)
    
    min_dist = 10.0
    found_dist = False

    # compute the line to the object's center
    center_vec = center - joint
    center_vec_norm = np.linalg.norm(center_vec)
    assert center_vec_norm > 1e-6
    center_vec /= center_vec_norm

    # debug print TODO
    # print("Computing distance for a joint...")
    # in_square_count = 0

    for face in faces:
        # compute the intersection point
        inter = line_plane_intersection(joint, center_vec, face)

        # check if the intersection point is within the face
        if inter is not None and is_pt_in_rect(inter, face):
            # debug counter TODO
            # in_square_count += 1

            # compute the distance
            dist = np.linalg.norm(inter - joint)
            if dist < min_dist:
                min_dist = dist
            found_dist = True
    
    # debug print TODO
    # print(f"Found {in_square_count} intersections in square.")
    
    # assert found_dist, "No distance found."
    if found_dist: 
        return min_dist
    else:
        return None

    # return min_dist

In [36]:
# path handling
pose_root = h2o_root + f'seq_{n_frames_per_seq}_{mode}/'
hand_dir = pose_root + f'poses_hand_{mode}/'
obj_dir = pose_root + f'poses_obj_{mode}/'
dist_dir = pose_root + f'distances_{mode}/'
os.makedirs(dist_dir, exist_ok=True) # create the destination directory

# face indices according to h2o convention, but 0-indexed
faces_indices = [
        [0, 1, 2, 3], [4, 5, 6, 7], [0, 1, 5, 4], 
        [2, 3, 7, 6], [0, 3, 7, 4], [1, 2, 6, 5]
    ]

# for each sequence TODO
for seq_id in range(1, n_sequences+1):
    print("\n\nSequence:", seq_id, '\n') # TODO
    no_dist = 0

    # get poses of current sequence
    hand_poses = np.load(hand_dir + f'{seq_id:03d}.npy')
    obj_poses = np.load(obj_dir + f'{seq_id:03d}.npy')
    assert hand_poses.shape[0] == n_frames_per_seq
    assert hand_poses.shape[1] == 128
    assert obj_poses.shape[0] == n_frames_per_seq
    assert obj_poses.shape[1] == 64

    # prepare distance array
    dist_array = np.zeros((n_frames_per_seq, 42))

    # for each frame TODO
    for frame_id in range(0, n_frames_per_seq):
        # print("\n\nFrame:", frame_id, '\n') # TODO
        
        # get and format hand poses of current frame
        hand_poses_left = hand_poses[frame_id, 1:64].reshape((21, 3))
        hand_poses_right = hand_poses[frame_id, 65:].reshape((21, 3))

        # extract object center point
        obj_center = obj_poses[frame_id, 1:4]
        # extract object corner vertices
        obj_corner_vert = obj_poses[frame_id, 4:]
        obj_corner_vert = obj_corner_vert[:8*3]
        obj_corner_vert = obj_corner_vert.reshape((8, 3))
        # extract faces as (4,3) arrays from the corner vertices according to the face indices
        faces = []
        for face_indices in faces_indices:
            face = obj_corner_vert[face_indices]
            faces.append(face)

        # for each joint
        for j in range(21):
            # print("\nJoint:", j) # TODO
            # compute the line to the object's center
            # print("\nLeft") # TODO
            distance_left = dist_joint_bbox(hand_poses_left[j], obj_center, faces)
            # print("\nRight") # TODO
            # print("Point:", hand_poses_right[j]) # TODO
            # print("BBox", faces) # TODO
            distance_right = dist_joint_bbox(hand_poses_right[j], obj_center, faces)
            # print("Distance:", distance_right) # TODO

            if distance_left is None:
                no_dist += 1
            if distance_right is None:
                no_dist += 1

            # write distance to array TODO
            # dist_array[frame_id, j] = distance_left
            # dist_array[frame_id, j+21] = distance_right

    # TODO 
    print("No distances found:", no_dist)
    # save the distance array

# output array
# for each sequence a file
# array (n_sequences, 42)




Sequence: 1 

No distances found: 0


Sequence: 2 

No distances found: 0


Sequence: 3 

No distances found: 0


Sequence: 4 

No distances found: 0


Sequence: 5 

No distances found: 13


Sequence: 6 

No distances found: 64


Sequence: 7 

No distances found: 86


Sequence: 8 

No distances found: 86


Sequence: 9 

No distances found: 86


Sequence: 10 

No distances found: 81


Sequence: 11 

No distances found: 77


Sequence: 12 

No distances found: 92


Sequence: 13 

No distances found: 0


Sequence: 14 

No distances found: 10


Sequence: 15 

No distances found: 29


Sequence: 16 

No distances found: 31


Sequence: 17 

No distances found: 45


Sequence: 18 

No distances found: 0


Sequence: 19 

No distances found: 37


Sequence: 20 

No distances found: 34


Sequence: 21 

No distances found: 156


Sequence: 22 

No distances found: 91


Sequence: 23 

No distances found: 44


Sequence: 24 

No distances found: 57


Sequence: 25 

No distances found: 57


Sequence: 26

KeyboardInterrupt: 

In [32]:
arr = np.array([1, 2, 3], dtype=np.double)
r = np.linalg.norm(arr)
print(type(r))

<class 'numpy.float64'>
